# Lecture 6: Risk Measures

[intro]

The main topics of discussion are:

 1. Basics of risk measurement
 2. Axiomization of risk measure theory
 3. Important risk measures
 4. Standard techniques for calculating VaR
 


In [1]:
#Importing standard libraries

import numpy as np
from matplotlib import pyplot as plt 
import scipy as sci
import pandas as pd